In [ ]:
pip install cmake==3.17.2

In [ ]:
pip install dlib==19.18.0

In [ ]:
pip install opencv-python==4.6.0.66

In [ ]:
pip install Pillow==9.2.0

In [ ]:
pip install face_recognition==1.3.0

In [ ]:
import tkinter as tk
import cv2
from PIL import ImageTk, Image
import threading
from keras.models import load_model
import numpy as np

# Biến kiểm tra trạng thái camera
camera_active = True

# Load mô hình đã train
model = load_model('selfie_train.h5')

# Hàm lấy hình từ camera và cập nhật lên giao diện
def update_image():
    # Đường dẫn camera, 0 là camera mặc định
    cap = cv2.VideoCapture(0)

    # Lấy kích thước khung hình
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

    while camera_active:
        # Đọc khung hình từ camera
        ret, frame = cap.read()

        if ret:
            # Chuyển đổi khung hình từ BGR sang RGB
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Chuyển đổi khung hình thành đối tượng Image
            image = Image.fromarray(rgb_frame)

            # Thay đổi kích thước ảnh nếu cần
            image = image.resize((400, 400))

            # Chuyển đổi ảnh thành đối tượng PhotoImage
            photo = ImageTk.PhotoImage(image)

            # Cập nhật ảnh trên giao diện
            window.image_label.config(image=photo)
            window.image_label.image = photo

    # Giải phóng camera
    cap.release()

    import tkinter as tk

# Tạo cửa sổ chào mừng
def create_welcome_window(username):
    welcome_window = tk.Toplevel(window)
    welcome_window.title("Chào mừng")
    welcome_label = tk.Label(welcome_window, text=f"Chào mừng {username}!")
    welcome_label.pack()


# Hàm xử lý sự kiện khi nút "LOGIN" được nhấn
def login():
    # Chụp ảnh từ camera
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()

    if ret:
        # Lưu ảnh với tên người đăng nhập
        username = window.entry.get()
        image_path = username + ".jpg"
        cv2.imwrite(image_path, frame)

        # Hiển thị ảnh mới lưu lên giao diện
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (400, 400))
        photo = ImageTk.PhotoImage(Image.fromarray(image))
        window.image_label.config(image=photo)
        window.image_label.image = photo

        # Nhận dạng ảnh
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (30, 40))
        from keras.utils.image_utils import img_to_array
        image = img_to_array(image)
        Ai = {1: 'An', 2: 'Kiet', 3: 'Long', 4: 'Minh'}
        image = image.reshape((1, 30, 40, 3))
        image = image.astype('float32')
        image = image / 255.0
        result = np.argmax(model.predict(image), axis=1)
        print(result)
        try:
            predicted_label = Ai[result[0]]
            print(predicted_label)
            if predicted_label == 'An':
                result_label_text = "Kết quả: Đăng nhập thành công!"
                create_welcome_window(username)  # Tạo cửa sổ chào mừng với tên người đăng nhập
            else:
                result_label_text = "Kết quả: Đăng nhập không thành công!"
        except KeyError:
            result_label_text = "Kết quả: Đăng nhập không thành công!"

        window.result_label.config(text=result_label_text)

    # Giải phóng camera
    cap.release()

# Hàm xử lý sự kiện khi nhấn phím
def key_pressed(event):
    if event.keysym == "Escape":
        global camera_active
        camera_active = False

# Tạo cửa sổ
window = tk.Tk()
window.title("Đăng Nhập")

# Tạo và hiển thị Label chứa ảnh
window.image_label = tk.Label(window)
window.image_label.pack()

# Tạo một luồng riêng để lấy hình từ camera và cập nhật lên giao diện
thread = threading.Thread(target=update_image)
thread.daemon = True
thread.start()

# Tạo Label và Entry để nhập tên người đăng nhập
username_label = tk.Label(window, text="Tên người đăng nhập:")
username_label.pack()
window.entry = tk.Entry(window)
window.entry.pack()

# Tạo nút nhấn "LOGIN"
login_button = tk.Button(window, text="LOGIN", command=login)
login_button.pack()
# Tạo và hiển thị Label để hiện kết quả
window.result_label = tk.Label(window, text="")
window.result_label.pack()

# Gán sự kiện khi nhấn phím
window.bind("<Key>", key_pressed)

# Khởi chạy giao diện
window.mainloop()